# NHS SitReps

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os
from datetime import datetime, timedelta

import csv
from xlrd import open_workbook, xldate_as_tuple

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

import common_core
import nhs_core
import nhs_download

## Configuration

Data to download from the NHS statistical work area

In [2]:
MAX_COLS_WITH_HEADERS = 6

REGION_HEADING = "NHS England Region"
CODE_HEADING = "Code"
NAME_HEADING = "Name"

CC_SHEET_NAME = "Adult critical care"
CC_CHART_TITLE = "Adult Critical Care Beds"

CC_OPEN_HEADING = "CC Adult Open"
CC_AVAIL_HEADING = "CC Adult avail"
CC_OCC_HEADING = "CC Adult Occ"

CC_OPEN_FIELD = "cc_open"
CC_OCC_FIELD = "cc_occupied"

DATE_FIELD = "date"

Y_MIN_FACTOR = 0
Y_MAX_FACTOR = 1.1

MIN_YEAR = 2011
MAX_YEAR = 2020

UEC_SITREPS = "uec-sitreps"

In [3]:
legacyTrustCodes = \
{
    "RDD": {"name": "Basildon And Thurrock University Hospitals NHS Foundation Trust", "nhsregion": "East of England"},
    "RC1": {"name": "Bedford Hospital NHS Trust", "nhsregion": "East of England"},
    "RLU": {"name": "Birmingham Women's NHS Foundation Trust", "nhsregion": "Midlands"},
    "RJF": {"name": "Burton Hospitals NHS Foundation Trust", "nhsregion": "Midlands"},
    "RR1": {"name": "Heart Of England NHS Foundation Trust", "nhsregion": "Midlands"},
    "RQQ": {"name": "Hinchingbrooke Health Care NHS Trust", "nhsregion": "East of England"},
    "RGQ": {"name": "Ipswich Hospital NHS Trust", "nhsregion": "East of England"},
    "RQ8": {"name": "Mid Essex Hospital Services NHS Trust", "nhsregion": "East of England"},
    "RW3": {"name": "Central Manchester University Hospitals NHS Foundation Trust", "nhsregion": "North West"},
    "RLN": {"name": "City Hospitals Sunderland NHS Foundation Trust", "nhsregion": "North East and Yorkshire"},
    "RNL": {"name": "North Cumbria University Hospitals NHS Trust", "nhsregion": "North East and Yorkshire"},
    "RQ6": {"name": "Royal Liverpool And Broadgreen University Hospitals NHS Trust", "nhsregion": "North West"},
    "RE9": {"name": "South Tyneside NHS Foundation Trust", "nhsregion": "North East and Yorkshire"},
    "RM2": {"name": "University Hospital Of South Manchester NHS Foundation Trust", "nhsregion": "North West"},
    "RD3": {"name": "Poole Hospital NHS Foundation Trust", "nhsregion": "South West"},
    "RBA": {"name": "Taunton And Somerset NHS Foundation Trust", "nhsregion": "South West"},
    "RDZ": {"name": "The Royal Bournemouth And Christchurch Hospitals NHS Foundation Trust", "nhsregion": "South West"},
    "RA3": {"name": "Weston Area Health NHS Trust", "nhsregion": "South West"},
    
    "RFW": {"name": "West Middlesex University Hospital NHS Trust", "nhsregion": "London"},
    "RVL": {"name": "Barnet And Chase Farm Hospitals NHS Trust", "nhsregion": "London"},
    "RC3": {"name": "Ealing Hospital NHS Trust", "nhsregion": "London"},
    "RV8": {"name": "North West London Hospitals NHS Trust", "nhsregion": "London"},
    "RJD": {"name": "Mid Staffordshire NHS Foundation Trust", "nhsregion": "Midlands"},
    "RD7": {"name": "Heatherwood And Wexham Park Hospitals NHS Foundation Trust", "nhsregion": "South East"},
    "RYQ": {"name": "SOUTH LONDON HEALTHCARE NHS TRUST", "nhsregion": "London"},
    "RM4": {"name": "TRAFFORD HEALTHCARE NHS TRUST", "nhsregion": "North West"},
    "RCC": {"name": "SCARBOROUGH AND NORTH EAST YORKSHIRE HEALTH CARE NHS TRUST", "nhsregion": "North East and Yorkshire"},
    "RGC": {"name": "WHIPPS CROSS UNIVERSITY HOSPITAL NHS TRUST", "nhsregion": "London"},
    "RNH": {"name": "NEWHAM UNIVERSITY HOSPITAL NHS TRUST", "nhsregion": "London"},
    "RNJ": {"name": "BARTS AND THE LONDON NHS TRUST", "nhsregion": "London"},
    "5QT": {"name": "ISLE OF WIGHT NHS PCT", "nhsregion": "South East"},
    "RN1": {"name": "WINCHESTER AND EASTLEIGH HEALTHCARE NHS TRUST", "nhsregion": "South East"},
    "5CQ": {"name": "MILTON KEYNES PCT", "nhsregion": "East of England"},
    "5QF": {"name": "BERKSHIRE WEST PCT", "nhsregion": "South East"},
    "RBF": {"name": "NUFFIELD ORTHOPAEDIC CENTRE NHS TRUST", "nhsregion": "South East"}
}

   ## Common Functions

In [4]:
def getCsvPath(year, areaType):
    """Determine CSV path for SitRep"""

    csvPath = os.path.join(common_core.dataDir, nhs_core.NHS_STATISTICS, "csv", "weekly", "sitreps", str(year), areaType)

    return csvPath

   ## XLSX to CSV

In [5]:
def findRowNos(sheet, heading):
    '''Find rows with the specified heading.'''

    rowNos = []

    for rowNo in range(sheet.nrows):
        for colNo in range(MAX_COLS_WITH_HEADERS):
            cellValue = sheet.cell(rowNo, colNo).value

            if isinstance(cellValue, str) and cellValue.startswith(heading):
                rowNos.append(rowNo)

    return rowNos


def findColNos(sheet, rowNo, heading, aliases = []):
    '''Determine the columns of headings from the cells in the specified row.'''

    colNos = []

    for colNo in range(sheet.ncols):
        cellValue = sheet.cell(rowNo, colNo).value

        if cellValue == heading:
            colNos.append(colNo)
        else:
            for alias in aliases:
                if cellValue == alias:
                    colNos.append(colNo)
                    break

    return colNos


def findReportDates(sheet, rowNo, dateMode):
    """Find all of the dates in the sheet"""
    
    reportDates = []

    for colNo in range(sheet.ncols):
        cellValue = sheet.cell(rowNo, colNo).value

        if isinstance(cellValue, str):
            if len(cellValue) > 0:
                try:
                    valueLen = len(cellValue)
                    if valueLen == 11:
                        # e.g. "01-NOV-2010"
                        dt = datetime.strptime(cellValue, '%d-%b-%Y')
                    elif valueLen == 12:
                        # e.g. "04-06-Dec-15"
                        dt = datetime.strptime(cellValue[:2] + cellValue[5:12], '%d-%b-%y')
                    elif valueLen == 16:
                        # Two possible formats to handle
                        if " " in cellValue:
                            # e.g. "16 to 18/11/2012" in 2012
                            dt = datetime.strptime(cellValue[:2] + cellValue[8:], '%d/%m/%Y')
                        else:
                            # e.g. "30-Jan-01-Feb-15"
                            dt = datetime.strptime(cellValue[:6] + cellValue[-3:], '%d-%b-%y')
                    elif valueLen == 19 or valueLen == 21:
                        # e.g. "31-Dec-15-03-Jan-16" or "29-Nov-13 - 01-Dec-13"
                        dt = datetime.strptime(cellValue[:9], '%d-%b-%y')
                    else:
                        raise RuntimeError(f"Unrecognised date format for {cellValue}")
                except:
                    print(f"Error converting {cellValue} to date")
                    raise

                reportDate = dt.strftime("%Y-%m-%d")
                reportDates.append(reportDate)
        else:
            if int(cellValue) > 0:
                year, month, day, hour, minute, second = xldate_as_tuple(cellValue, dateMode)
                reportDate = f"{year:04}-{month:02}-{day:02}"
                reportDates.append(reportDate)

    return reportDates
    

def scanSheet(trusts, sheet, dateMode=0, latest=False):
    """Scan all rows to detect regions, codes and names"""
    
    rowNo = findRowNos(sheet, CODE_HEADING)[0]

    regionColNos = findColNos(sheet, rowNo, REGION_HEADING)
    if len(regionColNos) > 0:
        regionColNo = regionColNos[0]
    else:
        regionColNo = None
        
    codeColNo = findColNos(sheet, rowNo, CODE_HEADING)[0]
    nameColNo = findColNos(sheet, rowNo, NAME_HEADING)[0]

    reportDates = findReportDates(sheet, rowNo - 1, dateMode)
    
    ccOpenColNos = findColNos(sheet, rowNo, CC_OPEN_HEADING, [CC_AVAIL_HEADING, CC_AVAIL_HEADING + "1"])
    assert len(reportDates) == len(ccOpenColNos), f"Mismatch of dates and CC beds open - {len(reportDates)} vs {len(ccOpenColNos)}"

    ccOccColNos = findColNos(sheet, rowNo, CC_OCC_HEADING, [CC_OCC_HEADING + "2"])
    assert len(reportDates) == len(ccOccColNos), f"Mismatch of dates and CC beds occupied - {len(reportDates)} vs {len(ccOccColNos)}"

    for rowNo in range(rowNo, sheet.nrows):
        code = sheet.cell(rowNo, codeColNo).value
        
        if len(code) == 3:
            name = sheet.cell(rowNo, nameColNo).value
            if regionColNo is not None:
                nhsRegion = sheet.cell(rowNo, regionColNo).value
            else:
                nhsRegion = None

            if code not in trusts:
                if latest == True:
                    trusts[code] = {"name": name, "nhsregion": nhsRegion}
                else:
                    if code in legacyTrustCodes:
                        trusts[code] = {"name": name, "nhsregion": legacyTrustCodes[code]["nhsregion"]}
                    else:
                        print(f"WARNING: Unrecognised code {code} for {name} - {sheet.cell(rowNo, 1).value}")
                        trusts[code] = {"name": name, "nhsregion": "unknown"}
                
    return codeColNo, reportDates, ccOpenColNos, ccOccColNos
   

def saveCsv(year, areaType, areaName, reportDates, ccOpen, ccOcc):
    """Save area to CSV"""

    # Ensure CSV path exists
    csvPath = getCsvPath(year, areaType)
    if not os.path.exists(csvPath):
        os.makedirs(csvPath)

    # Determine safe filename
    csvFn = os.path.join(csvPath, common_core.getSafeName(areaName) + ".csv")

    # Save data to CSV
    header = f"{DATE_FIELD},{CC_OPEN_FIELD},{CC_OCC_FIELD}"
    data = np.column_stack((reportDates, ccOpen, ccOcc))   
    np.savetxt(csvFn, data, fmt='%s', delimiter=',', header=header, comments='')
    
    
def loadSitRep(trusts, year, fileName, latest = False):
    """Load SitReps data into trusts"""

    partName = common_core.getPartName(fileName)

    print(f"Processing {year} - {partName}...")
    
    workbook = open_workbook(fileName)
    
    for sheet in workbook.sheets():           
        if sheet.name == CC_SHEET_NAME:
            codeColNo, reportDates, ccOpenColNos, ccOccColNos = scanSheet(trusts, sheet, workbook.datemode, latest = latest)
            
            numDates = len(reportDates)
            totalOpen = np.zeros(numDates, dtype=int)
            totalOcc = np.zeros(numDates, dtype=int)

            for nhsRegion in common_core.nhsRegionNames:
                regionOpen = np.zeros(numDates, dtype=int)
                regionOcc = np.zeros(numDates, dtype=int)
                
                for rowNo in range(sheet.nrows):
                    trustCode = sheet.cell(rowNo, codeColNo).value
                    
                    if trustCode in trusts:
                        trust = trusts[trustCode]

                        if trust["nhsregion"] == nhsRegion:
                            for i in range(len(ccOpenColNos)):
                                ccOpen = sheet.cell(rowNo, ccOpenColNos[i]).value
                                if ccOpen not in ["-", "N/A", ""]:
                                    ccOpen = int(ccOpen)

                                    # Check for ridiculous jumps in beds open - e.g. 18 to 547 for RWW in 2014/15
                                    if i > 0 and i < len(ccOpenColNos) - 1:
                                        ccOpenPrev = sheet.cell(rowNo, ccOpenColNos[i - 1]).value
                                        ccOpenNext = sheet.cell(rowNo, ccOpenColNos[i + 1]).value
                                        if ccOpenPrev not in ["-", "N/A", ""] and ccOpenNext not in ["-", "N/A", ""]:
                                            ccOpenPrev = int(ccOpenPrev)
                                            ccOpenNext = int(ccOpenNext)
                                            # Is the number of open beds is greater than the week before and the week after?
                                            if ccOpenPrev > 0 and ccOpen > ccOpenPrev * 4 and ccOpen > ccOpenNext * 4:
                                                print(f"WARNING: Patched {trustCode} open in {year} where {ccOpenPrev} -> {ccOpen}")
                                                ccOpen = (ccOpenPrev + ccOpenNext) // 2

                                    regionOpen[i] += ccOpen
                               
                                ccOcc = sheet.cell(rowNo, ccOccColNos[i]).value
                                if ccOcc not in ["-", "N/A", ""]:
                                    ccOcc = int(ccOcc)
                                    regionOcc[i] += ccOcc

                                try:
                                    # Report exceptionally high occupancy - over 5% capacity
                                    # Note: 2016 has a few anomalies but relatively minor data entry issues
                                    if year <= 2016 and ccOcc > ccOpen + 24 or year > 2016 and ccOcc > ccOpen * 1.05:
                                        # 2017 has massive data issue for St. Barts on 10 Dec 2017 - 490/120 beds occupied
                                        if year == 2017 and trustCode == "R1H":
                                            # Adjust to assume full occupancy - e.g. 120/120 beds
                                            regionOcc[i] -= regionOcc[i] - regionOpen[i]
                                            print(f"WARNING: Patched {trustCode} occ in {year} where {ccOcc} > {ccOpen}")
                                        else:
                                            print(f"WARNING: Ignored {trustCode} occ in {year} where {ccOcc} > {ccOpen}")
                                except:
                                    pass

                totalOpen += regionOpen
                totalOcc += regionOcc

                saveCsv(year, "nhsregion", nhsRegion, reportDates, regionOpen, regionOcc)

            # Now do England
            
            nationOpen = np.zeros(numDates, dtype=int)
            nationOcc = np.zeros(numDates, dtype=int)
            
            for nation in nhs_core.nationNames:
                rowNo = findRowNos(sheet, nation.upper())[0]

                for i in range(len(ccOpenColNos)):
                    cellValue = sheet.cell(rowNo, ccOpenColNos[i]).value
                    if cellValue not in ["-", "N/A"]:
                        nationOpen[i] += cellValue

                    cellValue = sheet.cell(rowNo, ccOccColNos[i]).value
                    if cellValue not in ["-", "N/A"]:
                        nationOcc[i] += cellValue

                # 2016 totals for England are incorrect in a few places
                # 2017 needed to be patched for St. Barts - 10 Dec 2017
                if year > 2017:
                    assert (nationOpen == totalOpen).all(), "Regions do not add up to nation"
                    assert (nationOcc == totalOcc).all(), "Regions do not add up to nation"

                saveCsv(year, "nation", nation, reportDates, totalOpen, totalOcc)


def convertSitReps(skipHistory = False):
    """Process all available SitReps"""

    trusts = {}

    partNames = nhs_download.downloadSitReps(skipExisting=True)
    for partName in partNames:
        year = int(nhs_core.SITREPS_YEAR[:4])
        xlsxFn = os.path.join(common_core.dataDir, partName)
        loadSitRep(trusts, year, xlsxFn, latest = True)

    if not skipHistory:
        for sitRepsFile in nhs_download.sitRepsFiles:
            for year in range(MAX_YEAR - 1, MIN_YEAR - 1, -1):
                xlsxPath = os.path.join(nhs_core.rawPath, sitRepsFile[0], nhs_core.SITREPS_CATEGORY, f"{year}-{year + 1}")
                for basename in os.listdir(xlsxPath):
                    if not basename.startswith("~$"):
                        fileName = os.path.join(xlsxPath, basename)
                        loadSitRep(trusts, year, fileName, latest = False)

## Create Charts

In [6]:
def loadArea(areaType, areaName):
    """Load CSV for single area"""
    
    data = {}
    
    for year in range(MIN_YEAR, MAX_YEAR + 1):
        csvPath = getCsvPath(year, areaType)
        csvFn = os.path.join(csvPath, common_core.getSafeName(areaName) + ".csv")

        try:
            with open(csvFn, 'r') as f:
                reader = csv.reader(f, delimiter = ',')

                dtype = []
                colNames = next(reader)

                for i in range(len(colNames)):
                    colName = colNames[i]
                    if colName == DATE_FIELD:
                        dtype.append((colName, "U10"))
                    else:
                        dtype.append((colName, "u4"))

                yearData = np.genfromtxt(f, dtype=dtype, delimiter=",")
                
                data[year] = yearData

        except:
            print(f"Failed to load CSV data for {areaName}")
            raise

    return data


def getStartDay(data):
    """Determine the earliest date of the year"""
    
    minDay = 365
    
    for year in data:
        day = datetime.strptime(data[year][0]["date"], '%Y-%m-%d').timetuple().tm_yday
        if day < minDay:
            minDay = day
            
    return minDay
    

def plotRegion(fig, ax, areaName, areaData, minDay, field=CC_OPEN_FIELD):
    """Plot a single region"""

    if field == CC_OPEN_FIELD:
        category = "Open"
    elif field == CC_OCC_FIELD:
        category = "Occupied"
    else:
        category = "???"

    title = f'{CC_CHART_TITLE} {category} in {areaName} {min(areaData)}-{max(areaData) + 1}'
    ax.set_title(title)

    ax.set_ylabel('Number of beds')
    ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))

    ax.set_prop_cycle(plt.cycler("color", plt.cm.viridis(np.linspace(1, 0, MAX_YEAR - MIN_YEAR + 1))))

    minValues = {}
    maxValues = {}
    xMin = xMax = 0
    for year in areaData:

        # Calculate the 7 day MA using convolve - n.b. years prior to 2016 only reported Mon-Fri, hence 5 day average
        if year >= 2016:
            y_points = np.convolve(areaData[year][field], np.ones(7)/7, mode="valid")
        else:
            y_points = np.convolve(areaData[year][field], np.ones(5)/5, mode="valid")

        # Save min and max value
        minValues[year] = round(min(y_points))
        maxValues[year] = round(max(y_points))

        # Calculate x_points using difference between dates
        startDate = datetime.strptime(areaData[year][0][DATE_FIELD], '%Y-%m-%d')
        offset = startDate.timetuple().tm_yday - minDay
        x_points = []
        
        for currDate in areaData[year][DATE_FIELD]:          
            diff = datetime.strptime(currDate, '%Y-%m-%d') - startDate
            x_points.append(offset + diff.days)
        
        # Drop the first and last dates that aren't included in the MA - 3 days or 2 days at each end
        if year >= 2016:
            x_points = x_points[3:-3]
        else:
            x_points = x_points[2:-2]

        if min(x_points) < xMin:
            xMin = min(x_points)
        if max(x_points) > xMax:
            xMax = max(x_points)

        # Plot the result
        ax.plot(x_points, y_points, label = f"Beds {category.lower()} {year}/{year % 100 + 1}") #, color='green')

    ax.set_ylim(ymin=min(minValues.values()) * Y_MIN_FACTOR, ymax=max(maxValues.values()) * Y_MAX_FACTOR)

    # Generate date labels
    dateLabels = []
    minDate = datetime.strptime(f"{MIN_YEAR}-{minDay}", '%Y-%j')
    for i in range(xMax + 3):
        newDate = minDate + timedelta(days=i)
        dateLabels.append(newDate.strftime("%-d %b"))

    # Update x-axis to use desired interval
    ax.set_xlim(xmin=xMin - 2, xmax=xMax + 3)
    ax.set_xticks(np.arange(0, len(dateLabels), step=7))
    ax.set_xticklabels(dateLabels[::7])
    
    textStr = 'All plots are shown as a 7 day centered moving average'
    textStr += '\n\nSitRep reporting periods vary each year, typically starting in Nov or Dec'
    ax.text(0.02, 0.95, textStr, transform=ax.transAxes, horizontalalignment='left', verticalalignment='top')

    textStr = f'Created {datetime.now().strftime("%-d %b %y")}\n@Mike_aka_Logiqx'
    ax.text(0.98, 0.95, textStr, transform=ax.transAxes, horizontalalignment='right', verticalalignment='top', fontsize='small')

    textStr = 'Source: NHS Statistical Work Areas\nUrgent and Emergency Care Daily Situation Reports'
    ax.text(0.98, 0.05, textStr, transform=ax.transAxes, horizontalalignment='right', verticalalignment='bottom', fontsize='small')

    textStr = 'Maximum 7d MA\n'
    for year in [*maxValues.keys()][:5]:
        textStr += f'\n{year}/{year % 100 + 1} = {maxValues[year]:,}'
    ax.text(0.02, 0.05, textStr, transform=ax.transAxes, horizontalalignment='left', verticalalignment='bottom')
    
    textStr = '\n'
    for year in [*maxValues.keys()][5:]:
        textStr += f'\n{year}/{year % 100 + 1} = {maxValues[year]:,}'
    ax.text(0.13, 0.05, textStr, transform=ax.transAxes, horizontalalignment='left', verticalalignment='bottom')
        
    ax.legend(loc = 'lower center', borderaxespad = 1.5, ncol=2)
    

def plotRegions():
    """Plot all of the regions"""

    data = {}
    minDay = 365
    
    areas = [("nation", nhs_core.nationNames), ("nhsregion", common_core.nhsRegionNames)]
    
    for area in areas:
        areaType = area[0]
        for areaName in area[1]:
            areaData = loadArea(areaType, areaName)
        
            areaStartDay = getStartDay(areaData)
            if areaStartDay < minDay:
                minDay = areaStartDay

            data[areaName] = areaData
        
    nhsRegions = [common_core.ENGLAND, "London", "South East", "East of England",
         "Midlands", "South West", "North West", "North East and Yorkshire"]

    for nhsRegion in nhsRegions:
        fields = [CC_OPEN_FIELD, CC_OCC_FIELD]
        fig, axs = plt.subplots(len(fields), figsize=(16, 12), dpi=150)

        for i in range(len(fields)):
            field = fields[i]
            plotRegion(fig, axs[i], nhsRegion, data[nhsRegion], minDay, field=field)

        plt.subplots_adjust(hspace=0.2)
        plt.show()

        partName = os.path.join("docs", UEC_SITREPS, common_core.getSafeName(nhsRegion) + ".png")
        fileName = os.path.join(common_core.projdir, partName)
        
        print(f"Saving {partName}...")

        fig.savefig(fileName, bbox_inches='tight', facecolor='w')
        plt.close(fig)

In [7]:
def saveIndexPage():
    """Save HTML page as index to images"""

    indexPageDetails = \
    {
        "TITLE_TEXT": "UEC SitReps",
        "H1_TEXT": "Urgent and Emergency Care Daily Situation Reports",
        "H2_TEXT": "Adult Critical Care Beds Open + Occupied in England",
        "CUSTOM_CSS": "css/custom.css",
        "FURTHER_HREF": "."
    }
    fileName = os.path.join(common_core.projdir, "docs", UEC_SITREPS, "gallery.html")
    indexPage = common_core.IndexPage(fileName, indexPageDetails)

    areaGroups = \
    [
        ["England", "London"],
        ["South East", "East of England"],
        ["North West", "North East and Yorkshire"],
        ["Midlands", "South West"]
    ]

    for areaNames in areaGroups:
        galleryName = " + ".join(areaNames)
        galleryDetails = \
        {
            "P_TEXT": galleryName
        }
        gallery = indexPage.addGallery(galleryName, galleryDetails)

        for areaName in areaNames:
            fileName = os.path.join(common_core.projdir, "docs", UEC_SITREPS, common_core.getSafeName(areaName) + ".png")

            figureDetails = \
            {
                "ALT_TEXT": areaName,
                "CAPTION_TEXT": areaName
            }
            figure = gallery.addFigure(areaName, ".", fileName, figureDetails)
            figure.createThumb(suffix="_thumb")

    html = indexPage.saveHtml()

## Running Interactively

In [8]:
if __name__ == '__main__':

    convertSitReps(skipHistory = True)

    figures = plotRegions()
    
    saveIndexPage()